In [1]:
import pandas as pd
import glob
import numpy as np

# new files since april-12

path = '../../corona/COVID-19/csse_covid_19_data/csse_covid_19_daily_reports_us/*.csv'

files = glob.glob(path)

us_states = pd.read_csv('us_states.csv',sep=';',header=None,usecols=[0])
us_states_df = pd.DataFrame()

for f in files:
    date = f.split('/')[-1].split('.')[0]
    
    df = pd.read_csv(f,sep=',',usecols=[0,2,5,6])
    df['Last_Update'] = (pd.to_datetime(df['Last_Update']))
    df['date'] = pd.to_datetime([date] * len(df))

    df.rename(columns={'Province_State' : 'state',
                      'Confirmed' : 'confirmed',
                      'Deaths' : 'deceased'},inplace=True)
    df = pd.merge(df,us_states,left_on='state',right_on=0)
    df.drop(0,axis=1,inplace=True)
    df.set_index('date',inplace=True)

    
    us_states_df = pd.concat([us_states_df,df])
    
us_states_df

,state,Last_Update,confirmed,deceased
date,,,,
2020-07-20,Alabama,2020-07-21 04:38:59,68891,1291
2020-07-20,Alaska,2020-07-21 04:38:59,1950,18
2020-07-20,Arizona,2020-07-21 04:38:59,145183,2784
2020-07-20,Arkansas,2020-07-21 04:38:59,33927,363
2020-07-20,California,2020-07-21 04:38:59,397870,7770
...,...,...,...,...
2020-04-15,Virginia,2020-04-15 22:56:51,6500,195
2020-04-15,Washington,2020-04-15 22:56:51,10942,552
2020-04-15,West Virginia,2020-04-15 22:56:51,702,10


In [2]:
state_names = us_states_df['state'].unique()

state_dic = dict()

def process_state(state_name):
    
    state = (us_states_df[us_states_df['state'] == state_name]).copy()
    state['inc'] = state['confirmed'] - state['confirmed'].shift()
    state['dead_inc'] = state['deceased'] - state['deceased'].shift()
    state['factor'] = state['confirmed'] / state['confirmed'].shift()
    state['dead_factor'] = state['deceased'] / state['deceased'].shift()
    state_dic[state_name] = state


In [3]:
for s in state_names:
    process_state(s)
    
state_dic['New York']

,state,Last_Update,confirmed,deceased,inc,dead_inc,factor,dead_factor
date,,,,,,,,
2020-07-20,New York,2020-07-21 04:38:59,407326,32506,NaN,NaN,NaN,NaN
2020-05-12,New York,2020-05-13 03:32:30,338485,27284,-68841.0,-5222.0,0.830993,0.839353
2020-06-09,New York,2020-06-10 04:07:14,379482,30458,40997.0,3174.0,1.121119,1.116332
2020-05-17,New York,2020-05-18 02:32:26,350121,28232,-29361.0,-2226.0,0.922629,0.926916
2020-08-08,New York,2020-08-09 04:35:07,420345,32768,70224.0,4536.0,1.200571,1.160669
...,...,...,...,...,...,...,...,...
2020-08-26,New York,2020-08-27 04:30:22,431340,32921,566.0,3.0,1.001314,1.000091
2020-04-14,New York,2020-04-14 23:33:31,203020,10842,-228320.0,-22079.0,0.470673,0.329334
2020-06-08,New York,2020-06-09 03:33:19,378799,30417,175779.0,19575.0,1.865821,2.805479


In [4]:
# old data from files before april 12
old_state_dic = pd.read_pickle('us_state_dict_old.pkl')



In [5]:
#https://worldpopulationreview.com/states/
us_state_density = pd.read_csv('us_states_pop_density.csv')
us_state_density.set_index('State',inplace=True)

us_state_economy = pd.read_csv('us_states_economy.csv',header=None,sep=';',
                               index_col=[1],thousands=',')

sqr_mile_factor = 2.58998811

us_state_density['density'] *= sqr_mile_factor 

us_state_economy.drop([0,2,3,5],axis=1,inplace=True)
us_state_economy.rename(columns={4:'gdp'},inplace=True)
us_state_economy.index.name='state'


In [6]:
us_state_density

,rank,Pop,Growth,Pop2018,Pop2010,growthSince2010,Percent,density
State,,,,,,,,
California,1,39937489,0.0096,39557045,37320903,0.0701,0.1194,664.002245
Texas,2,29472295,0.0268,28701845,25242679,0.1676,0.0881,292.203495
Florida,3,21992985,0.0326,21299325,18845785,0.1670,0.0658,1062.220428
New York,4,19440469,-0.0052,19542209,19400080,0.0021,0.0581,1068.424744
Pennsylvania,5,12820878,0.0011,12807060,12711158,0.0086,0.0383,742.147884
Illinois,6,12659682,-0.0064,12741080,12840762,-0.0141,0.0379,590.580226
Ohio,7,11747694,0.0050,11689442,11539327,0.0181,0.0351,744.631424
Georgia,8,10736059,0.0206,10519475,9711810,0.1055,0.0321,483.478001
North Carolina,9,10611862,0.0220,10383620,9574293,0.1084,0.0317,565.317223


In [7]:
joined_state_dic = dict()

for k,v in old_state_dic.items():
    new_state =  pd.concat([v,state_dic[k]])
    new_state.index = (pd.to_datetime(new_state.index)).date
    new_state = new_state[['confirmed','deceased','inc','dead_inc',
                          'factor','dead_factor']]
    
    new_state['inc'] = new_state['confirmed'] - new_state['confirmed'].shift()
    new_state['dead_inc'] = new_state['deceased'] - new_state['deceased'].shift()
    new_state['factor'] = new_state['confirmed'] / new_state['confirmed'].shift()
    new_state['dead_factor'] = new_state['deceased'] / new_state['deceased'].shift()
    new_state['density'] = us_state_density.loc[k,'density']
    new_state['gdp'] = us_state_economy.loc[k,'gdp']
    new_state['pct_dead'] = 100 * new_state['deceased'] / new_state['confirmed']
    new_state['conf_per_M'] = new_state['confirmed'] / (us_state_density.loc[k,'Pop'] / 1e6)
    new_state['dead_per_M'] = new_state['deceased'] / (us_state_density.loc[k,'Pop'] / 1e6)
    new_state = new_state[new_state['deceased'] > 1]
    new_state.replace(np.inf,np.nan,inplace=True)
    
    joined_state_dic[k] = new_state


In [8]:
joined_state_dic['New York']

,confirmed,deceased,inc,dead_inc,factor,dead_factor,density,gdp,pct_dead,conf_per_M,dead_per_M
2020-03-14,525.0,2.0,104.0,2.0,1.247031,NaN,1068.424744,89076,0.380952,27.005521,0.102878
2020-03-15,732.0,3.0,207.0,1.0,1.394286,1.500000,1068.424744,89076,0.409836,37.653413,0.154317
2020-03-16,967.0,10.0,235.0,7.0,1.321038,3.333333,1068.424744,89076,1.034126,49.741598,0.514391
2020-03-17,1706.0,13.0,739.0,3.0,1.764219,1.300000,1068.424744,89076,0.762016,87.755085,0.668708
2020-03-18,2495.0,16.0,789.0,3.0,1.462485,1.230769,1068.424744,89076,0.641283,128.340525,0.823025
...,...,...,...,...,...,...,...,...,...,...,...
2020-08-26,431340.0,32921.0,566.0,3.0,1.001314,1.000091,1068.424744,89076,7.632262,22187.736314,1693.426223
2020-04-14,203020.0,10842.0,-228320.0,-22079.0,0.470673,0.329334,1068.424744,89076,5.340361,10443.163691,557.702595
2020-06-08,378799.0,30417.0,175779.0,19575.0,1.865821,2.805479,1068.424744,89076,8.029852,19485.075180,1564.622746
2020-07-31,415014.0,32689.0,36215.0,2272.0,1.095605,1.074695,1068.424744,89076,7.876602,21347.941760,1681.492355


In [9]:
import pickle

with open ('us_state_dict.pkl','wb') as f:
    pickle.dump(joined_state_dic,f,pickle.HIGHEST_PROTOCOL)
    